# Prototype Development

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

from utils import *
from plots import *
from pedestrian import PedestrianDataset
from prototype import PedestrianDemo

data_path = Path("..") / "data"
counts_path = data_path / "Pedestrian_Counting_System___2009_to_Present__counts_per_hour_.csv"
sensor_path = data_path / "Pedestrian_Counting_System_-_Sensor_Locations.csv"

Data we'll need:

In [ ]:
class Demo(PedestrianDataset, PedestrianDemo):
    pass

demo = Demo.load(counts_path, lat_long_path=sensor_path, debug=True)

In [60]:
demo.prototype()

In [ ]:
# TODO
# -- diagnose and fix issues with sounds count/ sensor time series plots when one/few plots displayed
# -- add titles with current filter values to each plot
# -- sort out margins
# -- address artefcts in plots
# -- add prev 4 week average to time series plots etc

## Comparing Sensors by Year

In [ ]:
import functools

# inputs
sensor_input = widgets.Dropdown(options=SENSORS)
year_input = widgets.Dropdown(options=YEARS)
month_input = widgets.Dropdown(options=MONTHS)

# binding inputs to output function
out = widgets.interactive_output(
    display_output(functools.partial(plot_years, foot_df)),
    {"sensor": sensor_input, "month": month_input},
)

# layout
widgets.HBox(
    [
        widgets.VBox(
            [
                widgets.HTML("<H1>Comparing Sensors by Year</h1>"),
                sensor_input,
                month_input,
            ]
        ),
        out,
    ]
)

Ideas:
* align to eg first Monday of the month
* get rid of the 2000 from the initial x value see `tickformatstops` example near bottom of [this page](https://plotly.com/python/time-series/)
* extend to `sensor_plot`
* add ability to overlay mean/median lines from custom date (ranges)
* work out how to combine `plot_sensors` and `plot_year` into single `plot_comparison` that also allows selecting month, and even day?
* improve hover label

## Ipywidgets

* input controls
* display outputs
* binding inputs and outputs to interactions

In [ ]:
import ipywidgets as widgets

In [ ]:
widgets.Dropdown(options=["cat", "dog"])

In [ ]:
widgets.Dropdown(options=[1,2,3,4, 5])

In [ ]:
widgets.IntRangeSlider(
    value=[2009, 2020],
    min=2009,
    max=2020,
)